In [2]:
import pandas as pd

In [11]:
bd = pd.read_excel('Base de dato Apuestas Deportivas.xlsx')

In [12]:
bd

,Fecha,Local,Visitante,Goles L,Goles V,Tiros L,Tiros V,Tiros OT L,Tiros OT V,Faltas L,Faltas V,Tarjetas L,Tarjetas V,Offsides L,Offsides V,Corners L,Corners V
0,2024-01-10,WOLVES,BURNLEY,1,0,7,12,4,3,11,15,2,4,2,0,5,8
1,2024-01-10,LUTON,ARSENAL,3,4,6,23,4,9,14,7,2,1,1,4,3,8
2,2024-01-10,SHEFFIELD,LIVERPOOL,0,2,6,15,1,8,12,8,2,3,1,4,2,12
3,2024-01-10,FULHAM,NOTTINGHAM,5,0,14,4,6,1,7,10,0,1,2,3,6,1
4,2024-01-10,CRYSTAL PALACE,BOURNEMOUTH,0,2,16,11,3,4,10,17,1,3,0,0,3,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,2024-02-26,MAN UNITED,FULHAM,1,2,21,17,9,5,6,11,2,5,2,1,10,9
113,2024-02-26,BRIGTHON,EVERTON,1,1,23,6,7,3,9,14,2,3,1,3,11,3
114,2024-02-26,BOURNEMOUTH,MAN CITY,0,1,13,15,4,6,14,6,2,2,3,0,3,8
115,2024-02-26,ARSENAL,NEWCASTLE,4,1,18,3,8,2,15,6,1,0,3,2,9,0
